In [ ]:
%load_ext autoreload
%autoreload 2

%config IPCompleter.greedy=True

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
from astropy import units

import importlib

import simulation
import snapshot_obj
import simulation_tracing
import dataset_compute
import subhalo
import halo_matching

In [ ]:
importlib.reload(simulation)
importlib.reload(snapshot_obj)
importlib.reload(simulation_tracing)
importlib.reload(dataset_compute)
importlib.reload(subhalo)
importlib.reload(halo_matching)

# Plotting trajectories of halos

## Tracing a satellite

Get the simulation and define M31 and MW:

In [ ]:
sim = simulation.Simulation("V1_LR_fix")
sim.set_centrals((1,0),(2,0))

In [ ]:
sim.get_snapshot(120).index_of_halo(1,3)

Get the merger tree:

In [ ]:
matcher = halo_matching.SnapshotMatcher()
mtree = simulation_tracing.MergerTree(sim, matcher, branching="BackwardBranching")
snap_start = 101
snap_stop = 128
mtree.trace_all(snap_start, snap_stop)

First, identify the central halo through snapshots:

In [ ]:
gn = 1
central = subhalo.SubhaloTracer(sim, 127, gn, 0)

In [ ]:
central.trace(mtree)

Then, select a satellite that can be traced back through a couple snapshots (for instance, the M31 satellite (1,4)):

In [ ]:
satellite = subhalo.SubhaloTracer(sim, 110, idx_ref=14)
%time satellite.trace(mtree)

In [ ]:
for gn, sgn, sid in zip(satellite.get_halo_data('GroupNumber'),
                     satellite.get_halo_data('SubGroupNumber'),
                     satellite.tracer[1]):
    print("{}: ({}, {})".format(sid, gn, sgn))

In [ ]:
%time d = satellite.distance_to_central(central)
d = d * units.cm.to(units.kpc)

Compute the distance between the central and the satellite:

Plot in the xy- and xz-plane:

In [ ]:
fig,ax = plt.subplots(ncols=2, figsize=(12,6))

ax[0].set_title('gn{} sgn{}'.format(gn,sgn))

ax[0].plot(d[:,0], d[:,1], 'b')
ax[0].plot(d[:,0], d[:,1], 'ko')
ax[0].plot(d[0,0], d[0,1], 'ro', label='earliest')
ax[0].set_xlabel('$x$ [kpc]', fontsize=16)
ax[0].set_ylabel('$y$ [kpc]', fontsize=16)

ax[1].plot(d[:,0], d[:,2], 'b')
ax[1].plot(d[:,0], d[:,2], 'ko')
ax[1].plot(d[0,0], d[0,2], 'ro')
ax[1].set_xlabel('$x$ [kpc]', fontsize=16)
ax[1].set_ylabel('$z$ [kpc]', fontsize=16)

ax[0].legend()

# Save figure:

filename = 'satellite_trajectory_gn{}sgn{}_{}.png'.format(gn,sgn,sim.sim_id)

home = os.path.dirname(snapshot_obj.__file__)
path = os.path.join(home,"Figures",sim.sim_id)
print(path)

# If the directory does not exist, create it
if not os.path.exists(path):
    os.makedirs(path)
#plt.savefig(os.path.join(path,filename), dpi=200)

# Plot evolution of mass components

In [ ]:
snap_start = max(min(satellite.tracer[1]), min(central.tracer[1]))
snap_stop = min(max(satellite.tracer[1]), max(central.tracer[1])) + 1
print(snap_start, snap_stop)

In [ ]:
z = sim.get_redshifts(snap_start, snap_stop)
print(z)

In [ ]:
masses = satellite.get_halo_data("MassType") * units.g.to(units.Msun)

In [ ]:
fig,ax = plt.subplots(nrows=2, figsize=(10,8))

ax[0].set_title('gn{} sgn{}'.format(gn,sgn))

ax[0].set_yscale('log')
ax[0].set_ylabel('Mass $[\mathrm{M_\odot}]$', fontsize=16)

ax[1].set_xlabel('$z$', fontsize=16)
ax[1].set_ylabel('$r$ [kpc]', fontsize=16)

ax[0].yaxis.set_ticks_position('both')
ax[1].yaxis.set_ticks_position('both')

ax[0].invert_xaxis()
ax[1].invert_xaxis()

for pt,l in zip([0,1,4],['gas', 'dm', 'stars']):
    ax[0].plot(z, masses[:,pt], label=l)
    
ax[0].legend()

ax[1].plot(z, np.linalg.norm(d, axis=1))

# Save figure:

filename = 'satellite_mass_evol_gn{}sgn{}_{}.png'.format(gn,sgn,sim.sim_id)

home = os.path.dirname(snapshot_obj.__file__)
path = os.path.join(home,"Figures",sim.sim_id)

# If the directory does not exist, create it
if not os.path.exists(path):
    os.makedirs(path)
#plt.savefig(os.path.join(path,filename), dpi=200)